# Linux Kernel

# Netfilter
* [wikipedia](https://en.wikipedia.org/wiki/Netfilter)
* [A Deep Dive into Iptables and Netfilter Architecture](https://www.digitalocean.com/community/tutorials/a-deep-dive-into-iptables-and-netfilter-architecture) - 2022-11-02


> **iptables** is a generic firewalling software that allows you to define rulesets. Each rule within an IP table consists of a number of classifiers (iptables matches) and one connected action (iptables target).

> **nftables** is the successor of iptables, it allows for much more flexible, scalable and performance packet classification. This is where all the fancy new features are developed.

In [1]:
%env ROOT_PWD=zhoujiagen

env: ROOT_PWD=zhoujiagen


## 组件

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Netfilter-components.svg/2560px-Netfilter-components.svg.png)

### 内核模块

- ip_tables, ip6_tables, arp_tables, ebtables: hook系统中legacy包过滤组件.
- nf_tables: 新的包过滤组件.

### 用户空间工具

- iptables(8), ip6tables(8): iptables/ip6tables — administration tool for IPv4/IPv6 packet filtering and NAT
- ebtables(8): ebtables − Ethernet bridge frame table administration
- arptables(8): arptables − ARP table administration
- ipset(8): ipset — administration tool for IP sets
- nftables(8): nft - Administration tool of the nftables framework for packet filtering and classification
  - 替代iptables, ip6tables, arptables, ebtables

### hooks
- NF_IP_PRE_ROUTING: 在输入流量进入网络栈后很快触发, 在路由决策之前处理.
- NF_IP_LOCAL_IN: 输入包已路由且目的是本机后触发.
- NF_IP_FORWARD: 输入包已路由且转发到其他机器后触发.
- NF_IP_LOCAL_OUT: 在本机产生的输出流量达到网络栈后触发.
- NF_IP_POST_ROUTING: 在已路由后的输出或转发流量的实际发送之前触发.

## legacy iptables包过滤
* [iptables - wikipedia](https://en.wikipedia.org/wiki/Iptables)
* https://man7.org/linux/man-pages/man8/iptables.8.html
* https://man7.org/linux/man-pages/man8/iptables-extensions.8.html: iptables-extensions — list of extensions in the standard iptables distribution

数据流:
![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Netfilter-packet-flow.svg/2880px-Netfilter-packet-flow.svg.png)

### tables
- filter: 放行/拒绝过滤
- nat: 网络地址转换
- mangle: 修改IP头
- raw: 连接跟踪(connection tracking)
- security: 加SELinux标记

### chains
- PREROUTING: 由NF_IP_PRE_ROUTING hook触发
- INPUT: 由NF_IP_LOCAL_ING hook触发
- FORWARD: 由NF_IP_FORWARDG hook触发
- OUTPUT: 由NF_IP_LOCAL_OUTG hook触发
- POSTROUTING: 由NF_IP_POST_ROUTINGG hook触发

包通过的链:
- 收到的, 目的本机: PREROUTING -> INPUT
- 收到的, 目的其他机器: PREROUTING -> FORWARD -> POSTROUTING
- 本机产生的: OUTPUT -> POSTROUTING


### 表-链关系

| Tables↓/Chains→               | PREROUTING | INPUT | FORWARD | OUTPUT | POSTROUTING |
| :---------------------------- | :--------- | :---- | :------ | :----- | :---------- |
| (routing decision)            |            |       |         | ✓      |             |
| raw                           | ✓          |       |         | ✓      |             |
| (connection tracking enabled) | ✓          |       |         | ✓      |             |
| mangle                        | ✓          | ✓     | ✓       | ✓      | ✓           |
| nat (DNAT)                    | ✓          |       |         | ✓      |             |
| (routing decision)            | ✓          |       |         | ✓      |             |
| filter                        |            | ✓     | ✓       | ✓      |             |
| security                      |            | ✓     | ✓       | ✓      |             |
| nat (SNAT)                    |            | ✓     |         |        | ✓           |

### 规则
放置在table的chain中, chain被调用时, 依次匹配chain中的规则.

- 匹配部分(matching)
  - 协议类型, 目的/源地址, 目的/源端口, 目的/源网段, 接收/发送的网卡接口, 协议头, 连接状态等.
- 动作部分(action)
  - 终止目标(target): 控制转回hook
  - 非终止目标: 继续链的执行
    - 跳转目标: 跳转到其他链继续处理; 例跳转到用户自定义的链.

target:
- ACCEPT: 让包通过
- DROP: 丢弃包
- RETURN: 结束该链的遍历, 在前一个链(调用链)的下一个规则恢复执行
- user-defined chain's name: 在用户自定义的链执行

### examples
* [Iptables tutorial: securing VPS with Linux firewall](https://www.hostinger.com/tutorials/iptables-tutorial)
* [Iptables Essentials: Common Firewall Rules and Commands](https://www.digitalocean.com/community/tutorials/iptables-essentials-common-firewall-rules-and-commands)
* [iptables — a comprehensive guide](https://sudamtm.medium.com/iptables-a-comprehensive-guide-276b8604eff1)

In [5]:
# !echo $ROOT_PWD | sudo -S iptables --help

In [ ]:
# !man iptables > iptables.main

In [ ]:
# # installation
# !sudo apt-get update
# !sudo apt-get install iptables

# # Check the status of your current iptables configuration
# !sudo iptables -L -v

# # Define Chain Rules
# # -i (interface) — the network interface whose traffic you want to filter, such as eth0, lo, ppp0, etc.
# # -p (protocol) — the network protocol where your filtering process takes place. It can be either tcp, udp, udplite, icmp, sctp, icmpv6, and so on. Alternatively, you can type all to choose every protocol.
# # -s (source) — the address from which traffic comes from. You can add a hostname or IP address.
# # –dport (destination port) — the destination port number of a protocol, such as 22 (SSH), 443 (https), etc.
# # -j (target) — the target name (ACCEPT, DROP, RETURN). You need to insert this every time you make a new rule.
# # 
# # !sudo iptables -A <chain> -i <interface> -p <protocol (tcp/udp) > -s <source> --dport <port no.>  -j <target>

# # -i lo
# # To allow traffic on localhost
# !sudo iptables -A INPUT -i lo -j ACCEPT

# # -p
# # Enabling Connections on HTTP, SSH, and SSL Port
# !sudo iptables -A INPUT -p tcp --dport 22 -j ACCEPT
# !sudo iptables -A INPUT -p tcp --dport 80 -j ACCEPT
# !sudo iptables -A INPUT -p tcp --dport 443 -j ACCEPT

# # -s
# # Filtering Packets Based on Source
# !sudo iptables -A INPUT -s 192.168.1.3 -j ACCEPT
# !sudo iptables -A INPUT -s 192.168.1.3 -j DROP
# # drop packets from a range of IP addresses
# !sudo iptables -A INPUT -m iprange --src-range 192.168.1.100-192.168.1.200 -j DROP

# # Dropping all Other Traffic
# !sudo iptables -A INPUT -j DROP

# # Deleting Rules
# # remove all rules and start with a clean slate
# !sudo iptables -F

# # see all the available rules
# !sudo iptables -L --line-numbers

# # delete a specific rule
# !sudo iptables -D INPUT 3

# # Persist Changes
# !sudo iptables-save > /etc/iptables/rules.v4
# !sudo iptables-save > /etc/iptables/rules.v6
# # load the saved rules
# !sudo iptables-restore < /etc/iptables/rules.v4
# !sudo iptables-restore < /etc/iptables/rules.v6

# # automatic loading
# # still need to use !sudo iptables-save command each time you make changes to iptables
# !sudo apt-get install iptables-persistent

In [23]:
# 追加规则
!echo $ROOT_PWD | sudo -S iptables -A INPUT -p tcp --dport 81 -j ACCEPT

[sudo] password for zhoujiagen: 

In [25]:
# 删除规则
!echo $ROOT_PWD | sudo -S iptables -D INPUT 1

[sudo] password for zhoujiagen: 

In [24]:
# 查看所有规则
!echo $ROOT_PWD | sudo -S iptables -L -v --line-numbers

[sudo] password for zhoujiagen: Chain INPUT (policy ACCEPT 0 packets, 0 bytes)
num   pkts bytes target     prot opt in     out     source               destination         
1        0     0 ACCEPT     tcp  --  any    any     anywhere             anywhere             tcp dpt:81

Chain FORWARD (policy ACCEPT 0 packets, 0 bytes)
num   pkts bytes target     prot opt in     out     source               destination         

Chain OUTPUT (policy ACCEPT 0 packets, 0 bytes)
num   pkts bytes target     prot opt in     out     source               destination         


## nftables
* [wikipedia](https://en.wikipedia.org/wiki/Nftables)
* https://wiki.nftables.org/wiki-nftables/index.php/Main_Page
* https://manpages.ubuntu.com/manpages/jammy/man8/nft.8.html

In [ ]:
!sudo apt-get install nftables

In [36]:
!echo $ROOT_PWD | sudo -S nft --version

[sudo] password for zhoujiagen: nftables v1.0.2 (Lester Gooch)


In [39]:
# !echo $ROOT_PWD | sudo -S nft --help

In [34]:
!echo $ROOT_PWD | sudo -S nft list ruleset

[sudo] password for zhoujiagen: table ip filter {
	chain INPUT {
		type filter hook input priority filter; policy accept;
	}
}


In [40]:
!echo $ROOT_PWD | sudo -S nft list tables

[sudo] password for zhoujiagen: table ip filter


In [41]:
!echo $ROOT_PWD | sudo -S nft list chains

[sudo] password for zhoujiagen: table ip filter {
	chain INPUT {
		type filter hook input priority filter; policy accept;
	}
}


In [42]:
!echo $ROOT_PWD | sudo -S nft list sets

[sudo] password for zhoujiagen: table ip filter {
}


In [43]:
!echo $ROOT_PWD | sudo -S nft list maps

[sudo] password for zhoujiagen: table ip filter {
}


In [44]:
!echo $ROOT_PWD | sudo -S nft list flowtables

[sudo] password for zhoujiagen: table ip filter {
}
